In [1]:
import pandas as pd
from loguru import logger
import json
import joblib

## 1. Load Model and Artifacts from Training

In [9]:
retained_cols = json.load(open('../services/models/retained_cols.json'))
feat_cols = json.load(open('../services/models/feat_cols.json'))
scaler = joblib.load('../services/models/scaler.pkl')
model = joblib.load('../services/models/xgboost_clf_model.pkl')
label_col = 'Churn'

## 2. Load inference data

In [3]:
df_inference_raw = pd.read_csv('../data/inference/df_inference_raw.csv')
logger.info(f"Data loaded with shape: {df_inference_raw.shape}")

2024-10-10 23:21:26.598 | INFO     | __main__:<module>:2 - Data loaded with shape: (667, 11)


In [10]:
# actual inference data do not have labels
# we separate the labels for later use for evaluation only
labels = df_inference_raw[label_col].to_list()

# only keep features that were used in training
df_inference_cleaned= df_inference_raw[feat_cols]
df_inference_cleaned

,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
0,144,1,0,0.21,4,61.6,117,20.1,3.86,8.2
1,66,1,0,0.00,0,34.0,133,30.0,13.93,11.5
2,72,1,0,0.00,1,138.9,111,42.0,10.58,10.8
3,97,1,0,0.00,0,225.1,90,63.0,13.98,8.8
4,88,1,0,0.00,3,183.5,93,46.0,8.53,8.3
...,...,...,...,...,...,...,...,...,...,...
662,142,1,0,0.20,4,232.5,74,57.0,9.09,10.4
663,64,0,1,2.19,0,127.2,93,57.9,8.15,8.1
664,163,1,1,2.38,1,224.0,126,81.8,11.68,8.8
665,90,0,1,3.59,0,169.0,104,80.9,9.44,13.3


In [11]:
df_inference_featurized = df_inference_cleaned.copy(deep=True)
df_inference_featurized[feat_cols] = scaler.transform(df_inference_cleaned[feat_cols])

In [12]:
X_inference = df_inference_featurized
y_inference = labels

In [13]:
y_pred = model.predict(X_inference)

In [14]:
from sklearn.metrics import classification_report


print(classification_report(y_inference, y_pred))


              precision    recall  f1-score   support

           0       0.95      0.98      0.96       570
           1       0.85      0.69      0.76        97

    accuracy                           0.94       667
   macro avg       0.90      0.83      0.86       667
weighted avg       0.93      0.94      0.93       667

